---
title: "SASを用いた因果効果算出方法の確認"
author: "Kazuhisa Ishihara"
date: "2022年2月19日"
output: html_document
---

## SAS接続の確立

In [5]:
import saspy
import pandas as pd
from IPython.display import HTML

sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 15812



## Dataのインポート
いちいちpandasのDFを経由しなくてもいいかもしれないが、後でpythonでもやってみるため、とりあえず2段階で。

In [6]:
url= 'https://github.com/iwanami-datascience/vol3/raw/master/kato%26hoshino/q_data_x.csv'
df= pd.read_csv(url)
sasdata1 = sas.df2sd(df, 'data1')
sasdata1.describe()

,Variable,N,NMiss,Median,Mean,StdDev,Min,P25,P50,P75,Max
0,cm_dummy,10000.0,0.0,0.00000,0.414400,0.492643,0.00000,0.000000,0.00000,1.000000,1.000000
1,gamedummy,10000.0,0.0,0.00000,0.074000,0.261784,0.00000,0.000000,0.00000,0.000000,1.000000
2,area_kanto,10000.0,0.0,0.00000,0.091200,0.287908,0.00000,0.000000,0.00000,0.000000,1.000000
3,area_keihan,10000.0,0.0,1.00000,0.588700,0.492094,0.00000,0.000000,1.00000,1.000000,1.000000
4,area_tokai,10000.0,0.0,0.00000,0.111500,0.314766,0.00000,0.000000,0.00000,0.000000,1.000000
5,area_keihanshin,10000.0,0.0,0.00000,0.208600,0.406328,0.00000,0.000000,0.00000,0.000000,1.000000
6,age,10000.0,0.0,44.50000,40.841900,10.465434,19.00000,34.500000,44.50000,44.500000,60.000000
7,sex,10000.0,0.0,1.00000,0.640300,0.479936,0.00000,0.000000,1.00000,1.000000,1.000000
8,marry_dummy,10000.0,0.0,1.00000,0.647000,0.477927,0.00000,0.000000,1.00000,1.000000,1.000000
9,job_dummy1,10000.0,0.0,1.00000,0.566200,0.495623,0.00000,0.000000,1.00000,1.000000,1.000000


## 因果効果の算出
SASの場合、Causaltrtというプロシージャで因果効果の算出ができるということなので、確認も兼ね、この方法でも実施してみる。

In [7]:
result = sas.submit("""
proc causaltrt data = data1 all;
		class
		cm_dummy
		sex
		marry_dummy 
		child_dummy
		area_kanto
		area_tokai
		area_keihanshin
		job_dummy1 - job_dummy7
		fam_str_dummy1 - fam_str_dummy4
		;
		
	psmodel 
		cm_dummy(ref = "0") = 
		TVwatch_day
		age 
		sex 
		marry_dummy 
		child_dummy
		inc
		pmoney
		area_kanto
		area_tokai
		area_keihanshin
		job_dummy1 - job_dummy7
		fam_str_dummy1 - fam_str_dummy4
		;
	model gamedummy(ref = "0") /dist = bin;
run;
""")
print(result["LOG"])
HTML(result["LST"])



28                                                        SAS システム               2022年 2月26日 土曜日 03時01分00秒

10224      ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
10224    ! ods graphics on / outputfmt=png;
10225      
10226      
10227      proc causaltrt data = data1 all;
10228      		class
10229      		cm_dummy
10230      		sex
10231      		marry_dummy
10232      		child_dummy
10233      		area_kanto
10234      		area_tokai
10235      		area_keihanshin
10236      		job_dummy1 - job_dummy7
10237      		fam_str_dummy1 - fam_str_dummy4
10238      		;
10239      		
10240      	psmodel
10241      		cm_dummy(ref = "0") =
10242      		TVwatch_day
10243      		age
10244      		sex
10245      		marry_dummy
10246      		child_dummy
10247      		inc
10248      		pmoney
10249      		area_kanto
10250      		area_tokai
10251      		area_keihanshin
10252      		job_dummy1 - job_dummy7
10253      		fam_str_dummy1 - fam_str_

さすがのSAS、簡単です。
ただ、標準誤差がロバスト推定値となっており、書籍と違っている。SASのことだからどこかに出力に関するパラメータがあるんだろうけど今はわからない。
ちょっとProcedudre Guideを見ると、ブートストラップ等もSyntaxにあるから、あとでもう少し詳しく勉強してみるか。